# 0. 개념의 이해

나는 밥을 [ ] 에서 빈칸에 들어갈 말이 '먹는다'라는 것을 우리는 큰 고민 없이 알 수 있다. 밥은 통계적으로 먹는 것이고, 사람에 의해 먹히는 것이기 때문이다. 또 다른 예로 알바생이 커피를 [ ] 라면 아마도 만든다가 정답일 가능성이 높다. 알바생이 커피를 마실 수도 있지만, 통계적으론 만드니까!

인공지능이 글을 이해하게 하는 방식도 위와 같다. 어떤 문법적인 원리를 통해서가 아니고, 수많은 글을 읽게 함으로써 나는, 밥을, 그 다음이 먹는다 라는 사실을 알게 하는 것이다. 그런 이유에서 많은 데이터가 곧 좋은 결과를 만들어 낼 수 있다. 단어를 적재적소에 활용하는 능력이 발달된다고 생각할 수 있다.

이 방식을 가장 잘 처리하는 인공지능 중 하나가 순환신경망(RNN)으로, 이번 시간엔 자세한 내용보다는 간단한 구조를 중심으로 배우게 될 것이다.

앞에서 먹었다 를 만드는 법은 배웠지만, 가장 첫 시작인 나는 은 어떻게 만들어야 할까? 이는 start 라는 특수한 토큰을 맨 앞에 추가해 줌으로써 해결할 수 있다. 인공지능에게 "자, 이제 어떤 문장이든 생성해봐!" 라는 사인을 주는 셈이다. start 를 입력으로 받은 순환신경망은 다음 단어로 나는 을 생성하고, 생성한 단어를 다시 입력으로 사용한다. 이 순환적인 특성을 살려 순환신경망이라고 이름을 붙인다!

그렇게 순차적으로 밥을 먹었다 까지 생성하고 나면, 인공지능은 "다 만들었어!" 라는 사인으로 end 라는 특수한 토큰을 생성한다. 즉, 우리는 start 가 문장의 시작에 더해진 입력 데이터(문제지)와, end 가 문장의 끝에 더해진 출력 데이터(답안지)가 필요하며, 이는 문장 데이터만 있으면 만들어낼 수 있다는 것 또한 알 수 있다.

위 과정을 파이썬으로는 아래와 같이 작성한다.

In [1]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


# 1. 필요한 라이브러리 및 파일 불러들이기!!

In [2]:
# 가장 먼저 tensorflow 라이브러리를 불러들이고 버전 확인!!
import tensorflow as tf

print(tf.__version__)

2.8.2


In [3]:
import glob
import os, re 
import numpy as np

import pandas as pd

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어들임.
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/AIFFEL/lyrics/*'
txt_list = glob.glob(file_path)

raw_corpus = []

# glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 더 쉽다.
# lyrics 안에 있는 모든 txt 파일들을 읽은 후 raw_corpus 리스트에 문장 단위로 저장!!
# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

Mounted at /content/drive
데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


In [4]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.

    if idx > 9: break   # 문장 10개 확인
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


# 2. 데이터 전처리(정제)

입력된 문장을 전처리하는 과정을 거친다.

1. 소문자로 바꾸고, 양쪽 공백을 지운다.
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꾼다.
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다.
5. 다시 양쪽 공백을 지운다.
6. 문장 시작에는 start, 끝에는 end를 추가해 준다.(편의상 <>는 생략)

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                         # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)         # 2
    sentence = re.sub(r'[" "]+', " ", sentence)                 # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)       # 4
    sentence = sentence.strip()                                 # 5
    sentence = '<start> ' + sentence + ' <end>'                 # 6
    return sentence

# 이 엉터리 문장이 어떻게 필터링되는지 확인해 보면!!!
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


* coupus, 바로 여기에 정제된 문장을 모을 것이다.
* 또한 문장을 단어 단위로 토큰화를 하는 과정을 거치게 될 것인데, 토큰의 개수가 15개를 넘어가는 문장을 제외하는 과정을 거치기 위해 splitdata라는 변수를 지정했다. 그리고 splitdata는 공백을 제외한 문장의 길이, 즉 단어의 개수가 나오게 된다.
* 그 외에도 문장의 길이가 0인 문장 역시 건너뛴다.

In [6]:
#정제 데이터 구축하기
corpus = []

# 우리가 원하지 않는 문장 건너뜀
for sentence in raw_corpus:
    if len(sentence) == 0: continue           # 길이 0
    if len(sentence.split()) >= 13: continue  # 15개 이하(start,end포함)
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
#정제된 결과 10개만 추려서 확인
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>',
 '<start> had some bad love <end>']

* 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용할 것이다.
* 12000단어를 기억할 수 있는 tokenizer를 만든다. 단어장의 크기를 12000 이상으로 설정하라고 했기 때문에 num_words는 12000으로 설정한 것이다.
* 우리는 이미 문장을 정제했으니 filters가 필요 X
* 12000단어에 포함되지 못한 단어는 unk로 바꾸기
* corpus를 이용해 tokenizer 내부의 단어장을 완성한다.
* 준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다.
* 입력 데이터의 시퀀스 길이를 일정하게 맞춘다.
* 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다.
* 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용하면 된다.

In [7]:
#tokenize() 함수로 데이터를 Tensor로 변환
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, #단어장의 크기:12,000 이상 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)  # corpus를 Tensor로 변환 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 290  28 ...   0   0   0]
 [  2 223  13 ...   0   0   0]
 [  2  25  17 ...   0   0   0]
 ...
 [  2 263 192 ...   0   0   0]
 [  2 127   5 ...   0   0   0]
 [  2   7  36 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fba2cbf49d0>


In [8]:
print(tensor[:3, :10])

[[   2  290   28   96 4536    3    0    0    0    0]
 [   2  223   13   87  224    6  115    3    0    0]
 [   2   25   17 1072 2290    3    0    0    0    0]]


In [9]:
#단어사전 구축 인덱스 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

#생성된 텐서를 소스와 타겟으로 분리해 모델 학습
src_input = tensor[:, :-1]   #소스 문장을 생성
tgt_input = tensor[:, 1:]   #타겟 문장을 생성

print(src_input[0])
print(tgt_input[0])

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
[   2  290   28   96 4536    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[ 290   28   96 4536    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


* tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성.
* 마지막 토큰은 end가 아니라 pad일 가능성이 높을 것이다.
* src_input을 훈련 문장 셋으로 구성하고, tensor에서 start를 잘라내서 타겟 문장(tgt_input)을 생성.

In [10]:
#데이터셋 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만든다.
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 32), dtype=tf.int32, name=None), TensorSpec(shape=(256, 32), dtype=tf.int32, name=None))>

# 3. 평가 데이터셋 분리

In [11]:
# 총 데이터의 20% 를 평가 데이터셋
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 15)

print(enc_train.shape)
print(enc_val.shape)
print(dec_train.shape)

(127100, 32)
(31776, 32)
(127100, 32)


In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

# 이번엔 이 embedding_size와 hidden_size를 하이퍼 파라미터로써 사용하게 될 것이다!!    
embedding_size = 256     # 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024       # 모델에 얼마나 많은 일꾼을 둘 것인가
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 데이터셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-1.63065575e-04, -2.27829543e-04, -7.44723002e-05, ...,
         -4.18515207e-04, -1.27158972e-04,  3.54587479e-04],
        [-3.96343850e-04, -4.33233276e-04, -2.94095837e-04, ...,
         -6.80554425e-04, -1.94785796e-04,  7.86120421e-04],
        [-5.45721618e-04, -6.63791550e-04, -3.54479038e-04, ...,
         -1.27532787e-03, -2.76885927e-04,  9.73741873e-04],
        ...,
        [ 1.77094771e-03, -6.88493659e-04, -5.27805043e-03, ...,
          1.73789740e-04, -1.63608952e-03, -7.56788766e-04],
        [ 1.81339576e-03, -6.91739435e-04, -5.37482556e-03, ...,
          1.62553493e-04, -1.72821013e-03, -7.74133019e-04],
        [ 1.84905436e-03, -6.95305469e-04, -5.45659941e-03, ...,
          1.50050648e-04, -1.80373504e-03, -7.87228637e-04]],

       [[-1.63065575e-04, -2.27829543e-04, -7.44723002e-05, ...,
         -4.18515207e-04, -1.27158972e-04,  3.54587479e-04],
        [-3.63515981e-04, -6.15823432e-04,  1

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


이제 모델을 학습할 시간!!! 파라미터를 보면 알겠지만 무려 3000만개에 육박한다. 모델 학습하는 게 시간이 걸린다는 것을 알 수 있다. 아무리 빨라도 최소 15분 정도 걸린다고 하니 여유를 가지고 작업하자!!!  

model.fit에는 다양한 인자를 넣을 수 있다고 한다. 자세한 내용은 다음 사이트를 참고하면 좋을 듯 싶다.  
https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit 

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, 
          dec_train, 
          epochs=10,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
497/497 [==============================] - 122s 236ms/step - loss: 1.6420 - val_loss: 1.4055
Epoch 2/10
497/497 [==============================] - 117s 235ms/step - loss: 1.3589 - val_loss: 1.3311
Epoch 3/10
497/497 [==============================] - 116s 233ms/step - loss: 1.2873 - val_loss: 1.2808
Epoch 4/10
497/497 [==============================] - 116s 233ms/step - loss: 1.2325 - val_loss: 1.2419
Epoch 5/10
497/497 [==============================] - 116s 234ms/step - loss: 1.1845 - val_loss: 1.2116
Epoch 6/10
497/497 [==============================] - 117s 235ms/step - loss: 1.1441 - val_loss: 1.1865
Epoch 7/10
497/497 [==============================] - 116s 234ms/step - loss: 1.1014 - val_loss: 1.1655
Epoch 8/10
497/497 [==============================] - 119s 240ms/step - loss: 1.0634 - val_loss: 1.1482
Epoch 9/10
497/497 [==============================] - 117s 235ms/step - loss: 1.0277 - val_loss: 1.1322
Epoch 10/10
497/497 [==============================] - 116s 234m

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        # 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        # 예측된 값 중 가장 높은 확률인 word index를 뽑기
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 예측된 word index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [21]:
#문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna be <end> '

# 회고

이번 시간에는 자연어 처리 기법을 통해 문장 내 단어들을 토큰화하고 단어에 추상적 의미를 부여해 주는 워드 벡터로 바꿔주는 embedding 레이어가 포함된 모델을 설계하여 원하는 가사 한 줄을 써 보는 실습을 하게 되었다.  

처음 할 때는 주피터 노트북을 이용해서 실행을 하였지만, 똑같이 코드를 작성하였음에도 계속 원인을 모를 오류가 걸리고 주피터 노트북에서 batch_size를 조금 낮춰가며 엄청나게 많은 데이터셋의 수용력을 조금 늘려서 훈련 속도를 빠르게 해 줘 봤지만 터무니없이 느려서 혼돈이 오기도 했다.  

결국 이번 노드는 코랩을 통해서 실행하게 되었고, 정상적으로 코드가 잘 돌아갔으며 결과도 원하는 대로 나올 수 있었다. 좀 더 공부해서 주피터 노트북으로 했을 때 발생했던 문제점에 대해 해결해 보기도 하며 문제 해결 능력을 더 향상시키고자 한다.